**Functional Autoencoder (FAE):** 

https://github.com/CedricBeaulac/FAE/tree/main

In [1]:
# Import modules
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import skfda as fda
from skfda import representation as representation
from skfda.exploratory.visualization import FPCAPlot
import scipy
from scipy import stats
from scipy.interpolate import BSpline
import ignite
import os
import sklearn
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import random
from random import seed
import statistics
from statistics import stdev
from datetime import datetime
import matplotlib.ticker as mtick
from random import seed
import random

# Import functions
import os
import sys
from pathlib import Path
project_dir = Path(r"D:\Mobina\Marquette\FAE Codes")
os.chdir(project_dir)
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

import Functions
from Functions import *

In [2]:
# Read_ElNino_Data.py file for reading the data 
"""
This script contains the code for importing and pre-processing the simulation data sets
in the manuscript "Functional Autoencoder for Smoothing and Representation Learning".

@author: Sidi Wu
"""

import pandas as pd
import numpy as np
from numpy import *
import os
import torch

os.chdir(r"D:\Mobina\Marquette\FAE Codes")

#####################################
### Real application: ElNino data set
#####################################
# Import dataset
x_raw = pd.read_csv('Dataset/ElNino_ERSST.csv')
tpts_raw = pd.read_csv('Dataset/ElNino_ERSST_tpts.csv')
label_table = pd.read_csv('Dataset/ElNino_ERSST_label.csv')
label = label_table.x.to_numpy()
time_grid = np.array(tpts_raw).flatten()

# Pre-process Data sets
# Prepare numpy/tensor data
x_np = np.array(x_raw).astype(float)
x = torch.tensor(x_np).float()
x_mean = torch.mean(x,0)
x = x - torch.mean(x,0)

# Rescale timestamp to [0,1]
tpts_np = np.array(tpts_raw)
tpts_rescale = (tpts_np - min(tpts_np)) / np.ptp(tpts_np)
tpts = torch.tensor(np.array(tpts_rescale))
n_tpts = len(tpts)


In [ ]:
"""
Comparison of Linear Functional Autoencoder (FAE) and FPCA (implemented as PCA)
on the El Niño data, using the ideas from
"Functional Autoencoder for Smoothing and Representation Learning".

This version:
- Uses a purely linear autoencoder (no basis functions, no nonlinearity)
- Uses PCA on the same centered data as FPCA
- Provides side-by-side comparison (MSE + classification accuracy)
"""

# ============================
# Imports
# ============================
import os
import sys
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import optim

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
import random

# ============================
# 1. Load and preprocess data
# ============================

# --- Load data just like in your original script ---
os.chdir(r"D:\Mobina\Marquette\FAE Codes")

# Import dataset
x_raw = pd.read_csv('Dataset/ElNino_ERSST.csv')
tpts_raw = pd.read_csv('Dataset/ElNino_ERSST_tpts.csv')
label_table = pd.read_csv('Dataset/ElNino_ERSST_label.csv')

label = label_table.x.to_numpy()
time_grid = np.array(tpts_raw).flatten()

# N × T matrix
x_np = np.array(x_raw).astype(float)
x = torch.tensor(x_np).float()  # N × T

# Center data (very important for PCA/AE equivalence)
x_mean = torch.mean(x, dim=0)
x_centered = x - x_mean

# Also keep a numpy version
x_centered_np = x_centered.numpy()

n_subjects, n_time = x_centered_np.shape

# ============================
# 2. Utility functions
# ============================

def split_data(x_tensor, labels, split_rate=0.8, seed_no=0):
    """
    Split into train/test, return tensors and index of train subjects.
    """
    N = x_tensor.shape[0]
    idx = np.arange(N)
    train_idx, test_idx = train_test_split(
        idx, train_size=split_rate, random_state=seed_no, shuffle=True
    )
    TrainData = x_tensor[train_idx]
    TestData = x_tensor[test_idx]
    TrainLabel = labels[train_idx]
    TestLabel = labels[test_idx]
    return TrainData, TestData, TrainLabel, TestLabel, train_idx

def eval_mse_sdse(true_tensor, pred_tensor):
    """
    Compute:
    - Mean MSE over all subjects
    - Standard deviation of curve-wise MSEs
    """
    true_np = true_tensor.numpy()
    pred_np = pred_tensor.numpy()
    # Curve-wise MSE
    mse_curves = np.mean((true_np - pred_np) ** 2, axis=1)  # length N
    mse_mean = mse_curves.mean()
    mse_sd = mse_curves.std()
    return mse_mean, mse_sd

# ============================
# 3. Linear Autoencoder (FAE)
# ============================

class LinearFAE(nn.Module):
    """
    Purely linear autoencoder:
    - Encoder: Linear(input_dim -> n_rep), no bias (to match PCA more closely)
    - Decoder: Linear(n_rep -> input_dim), no bias
    """
    def __init__(self, input_dim, n_rep, weight_std=None):
        super().__init__()
        self.encoder = nn.Linear(input_dim, n_rep, bias=False)
        self.decoder = nn.Linear(n_rep, input_dim, bias=False)
        self.activation = nn.Identity()

        if weight_std is not None:
            nn.init.normal_(self.encoder.weight, mean=0.0, std=weight_std)
            nn.init.normal_(self.decoder.weight, mean=0.0, std=weight_std)

    def forward(self, x):
        # x: N × T
        rep = self.activation(self.encoder(x))  # N × n_rep
        out = self.decoder(rep)                 # N × T
        return out, rep

def train_fae(model, train_loader, optimizer, loss_function, device):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        batch = batch.to(device).float()
        optimizer.zero_grad()
        x_hat, rep = model(batch)
        loss = loss_function(x_hat, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

def pred_fae(model, data_tensor, loss_function, device):
    model.eval()
    with torch.no_grad():
        data = data_tensor.to(device).float()
        out, rep = model(data)
        loss = loss_function(out, data)
    return out.cpu(), rep.cpu(), loss.item()

# ============================
# 4. FPCA as PCA on grid data
# ============================

def fpca_fit_predict(TrainData, TestData, n_rep):
    """
    FPCA implemented as PCA on discretized curves (N × T).
    """
    Train_np = TrainData.numpy()
    Test_np = TestData.numpy()

    pca = PCA(n_components=n_rep)
    pca.fit(Train_np)

    # scores
    scores_train = pca.transform(Train_np)   # N_train × n_rep
    scores_test = pca.transform(Test_np)     # N_test × n_rep
    scores_all = pca.transform(x_centered_np)

    # reconstructions
    pred_train = pca.inverse_transform(scores_train)
    pred_test = pca.inverse_transform(scores_test)
    pred_all = pca.inverse_transform(scores_all)

    # back to torch tensors
    pred_train_t = torch.tensor(pred_train).float()
    pred_test_t = torch.tensor(pred_test).float()
    pred_all_t = torch.tensor(pred_all).float()

    return (pca,
            scores_train, scores_test, scores_all,
            pred_train_t, pred_test_t, pred_all_t)

# ============================
# 5. Experiment setup
# ============================

niter = 20
random.seed(743)
niter_seed = random.sample(range(5000), niter)

n_rep = 5            # latent dimension
epochs = 3000        # can reduce if too slow
batch_size = 28
init_weight_sd = 0.5
split_rate = 0.8

device = torch.device("cpu")
loss_function = nn.MSELoss()

# --- Storage for results ---
FAE_pred_train_acc_mean_niter = []
FAE_pred_train_acc_sd_niter = []
FAE_pred_test_acc_mean_niter = []
FAE_pred_test_acc_sd_niter = []
classification_FAE_train_niter = []
classification_FAE_test_niter = []

FPCA_pred_train_acc_mean_niter = []
FPCA_pred_train_acc_sd_niter = []
FPCA_pred_test_acc_mean_niter = []
FPCA_pred_test_acc_sd_niter = []
classification_FPCA_train_niter = []
classification_FPCA_test_niter = []

# To compare representations more explicitly (optional)
corr_first_component_niter = []

# ============================
# 6. Main loop over replicates
# ============================

for i in range(niter):
    print(f"\n========== Replicate {i+1}/{niter} ==========")
    seed_no = niter_seed[i]

    # ------------------------
    # Data split
    # ------------------------
    TrainData, TestData, TrainLabel, TestLabel, train_no = split_data(
        x_centered, label, split_rate=split_rate, seed_no=seed_no
    )

    # ------------------------
    # 6a. Train Linear FAE
    # ------------------------
    model = LinearFAE(input_dim=n_time, n_rep=n_rep, weight_std=init_weight_sd)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)

    train_loader = torch.utils.data.DataLoader(
        TrainData, batch_size=batch_size, shuffle=True
    )

    for epoch in range(1, epochs + 1):
        loss_epoch = train_fae(
            model=model,
            train_loader=train_loader,
            optimizer=optimizer,
            loss_function=loss_function,
            device=device,
        )
        if epoch % 500 == 0:
            # quick monitoring on test loss
            fae_pred_test, fae_reps_test, fae_test_loss, _ = None, None, None, None
            fae_pred_test, fae_reps_test, fae_test_loss = pred_fae(
                model, TestData, loss_function, device
            )[:3]
            print(f"Epoch {epoch}: train_loss={loss_epoch:.4f}, test_loss={fae_test_loss:.4f}")

    # final predictions
    FAE_pred_train, FAE_reps_train, FAE_train_loss = pred_fae(
        model, TrainData, loss_function, device
    )
    FAE_pred_test, FAE_reps_test, FAE_test_loss = pred_fae(
        model, TestData, loss_function, device
    )
    FAE_pred_all, FAE_reps_all, _ = pred_fae(
        model, x_centered, loss_function, device
    )

    # prediction accuracy (MSE)
    fae_train_mse_mean, fae_train_mse_sd = eval_mse_sdse(TrainData, FAE_pred_train)
    fae_test_mse_mean, fae_test_mse_sd = eval_mse_sdse(TestData, FAE_pred_test)

    FAE_pred_train_acc_mean_niter.append(fae_train_mse_mean)
    FAE_pred_train_acc_sd_niter.append(fae_train_mse_sd)
    FAE_pred_test_acc_mean_niter.append(fae_test_mse_mean)
    FAE_pred_test_acc_sd_niter.append(fae_test_mse_sd)

    # Classification using FAE representations
    FAE_classifier = LogisticRegression(
        solver='liblinear', random_state=0, multi_class='auto'
    ).fit(FAE_reps_train.numpy(), TrainLabel)

    classification_FAE_train_niter.append(
        FAE_classifier.score(FAE_reps_train.numpy(), TrainLabel)
    )
    classification_FAE_test_niter.append(
        FAE_classifier.score(FAE_reps_test.numpy(), TestLabel)
    )

    # ------------------------
    # 6b. FPCA as PCA on grid
    # ------------------------
    (fpca_model,
     fpc_scores_train, fpc_scores_test, fpc_scores_all,
     FPCA_pred_train, FPCA_pred_test, FPCA_pred_all) = fpca_fit_predict(
        TrainData, TestData, n_rep=n_rep
    )

    # prediction accuracy (MSE)
    fpca_train_mse_mean, fpca_train_mse_sd = eval_mse_sdse(TrainData, FPCA_pred_train)
    fpca_test_mse_mean, fpca_test_mse_sd = eval_mse_sdse(TestData, FPCA_pred_test)

    FPCA_pred_train_acc_mean_niter.append(fpca_train_mse_mean)
    FPCA_pred_train_acc_sd_niter.append(fpca_train_mse_sd)
    FPCA_pred_test_acc_mean_niter.append(fpca_test_mse_mean)
    FPCA_pred_test_acc_sd_niter.append(fpca_test_mse_sd)

    # Classification using FPCA scores
    FPCA_classifier = LogisticRegression(
        solver='liblinear', random_state=0, multi_class='auto'
    ).fit(fpc_scores_train, TrainLabel)

    classification_FPCA_train_niter.append(
        FPCA_classifier.score(fpc_scores_train, TrainLabel)
    )
    classification_FPCA_test_niter.append(
        FPCA_classifier.score(fpc_scores_test, TestLabel)
    )

    # ------------------------
    # 6c. Representation comparison (optional)
    # Compare first FAE component vs first FPCA component
    # (They are only identifiable up to rotation and sign,
    #  so correlation ~ ±1 indicates strong equivalence.)
    # Use all subjects:
    fae_first = FAE_reps_all.numpy()[:, 0]
    fpca_first = fpc_scores_all[:, 0]
    corr = np.corrcoef(fae_first, fpca_first)[0, 1]
    corr_first_component_niter.append(corr)
    print(f"Correlation between first FAE and FPCA component: {corr:.4f}")

print("\n================ SUMMARY ================\n")

# ============================
# 7. Aggregate results & stats
# ============================

FAE_train_mse_mean = np.mean(FAE_pred_train_acc_mean_niter)
FAE_train_mse_sd   = np.std(FAE_pred_train_acc_mean_niter)
FAE_test_mse_mean  = np.mean(FAE_pred_test_acc_mean_niter)
FAE_test_mse_sd    = np.std(FAE_pred_test_acc_mean_niter)

FPCA_train_mse_mean = np.mean(FPCA_pred_train_acc_mean_niter)
FPCA_train_mse_sd   = np.std(FPCA_pred_train_acc_mean_niter)
FPCA_test_mse_mean  = np.mean(FPCA_pred_test_acc_mean_niter)
FPCA_test_mse_sd    = np.std(FPCA_pred_test_acc_mean_niter)

FAE_train_acc_mean = np.mean(classification_FAE_train_niter)
FAE_train_acc_sd   = np.std(classification_FAE_train_niter)
FAE_test_acc_mean  = np.mean(classification_FAE_test_niter)
FAE_test_acc_sd    = np.std(classification_FAE_test_niter)

FPCA_train_acc_mean = np.mean(classification_FPCA_train_niter)
FPCA_train_acc_sd   = np.std(classification_FPCA_train_niter)
FPCA_test_acc_mean  = np.mean(classification_FPCA_test_niter)
FPCA_test_acc_sd    = np.std(classification_FPCA_test_niter)

print("--- Linear FAE Results ---")
print(f"Train MSE Mean: {FAE_train_mse_mean:.4f}; Train MSE SD: {FAE_train_mse_sd:.4f}")
print(f"Test  MSE Mean: {FAE_test_mse_mean:.4f}; Test  MSE SD: {FAE_test_mse_sd:.4f}")
print(f"Train Classification Acc Mean: {FAE_train_acc_mean:.4f}; SD: {FAE_train_acc_sd:.4f}")
print(f"Test  Classification Acc Mean: {FAE_test_acc_mean:.4f}; SD: {FAE_test_acc_sd:.4f}\n")

print("--- FPCA (PCA) Results ---")
print(f"Train MSE Mean: {FPCA_train_mse_mean:.4f}; Train MSE SD: {FPCA_train_mse_sd:.4f}")
print(f"Test  MSE Mean: {FPCA_test_mse_mean:.4f}; Test  MSE SD: {FPCA_test_mse_sd:.4f}")
print(f"Train Classification Acc Mean: {FPCA_train_acc_mean:.4f}; SD: {FPCA_train_acc_sd:.4f}")
print(f"Test  Classification Acc Mean: {FPCA_test_acc_mean:.4f}; SD: {FPCA_test_acc_sd:.4f}\n")

# Paired t-tests (per replicate)
print("--- Paired t-tests (per replicate) ---")
print("Prediction error (MSE) on test:")
tt_mse = stats.ttest_rel(FAE_pred_test_acc_mean_niter, FPCA_pred_test_acc_mean_niter)
print(f"t = {tt_mse.statistic:.4f}, p = {tt_mse.pvalue:.4e}")

print("Classification accuracy on test:")
tt_acc = stats.ttest_rel(classification_FAE_test_niter, classification_FPCA_test_niter)
print(f"t = {tt_acc.statistic:.4f}, p = {tt_acc.pvalue:.4e}")

print("\nMean correlation between first FAE and FPCA component across replicates:")
print(f"{np.mean(corr_first_component_niter):.4f} (SD {np.std(corr_first_component_niter):.4f})")



========== Replicate 1/20 ==========
Epoch 500: train_loss=0.6685, test_loss=0.0828
Epoch 1000: train_loss=0.3144, test_loss=0.0369
Epoch 1500: train_loss=0.2054, test_loss=0.0211
Epoch 2000: train_loss=0.1690, test_loss=0.0183
